In [6]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached webcolors-24.11.1-py3-none-any.whl.metadata (2.2 kB)
Using cached webcolors-24.11.1-py3-none-any.whl (14 kB)
Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
Using cached uri_template-1.3.0-py3-none-any.whl (11 kB)


In [8]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run script server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [9]:
import os

# Make sure 'data' directory exists
os.makedirs("data", exist_ok=True)

# Now save FAISS index and metadata
faiss.write_index(index, "data/faiss_index.idx")
with open("data/faiss_meta.json", "w", encoding='utf8') as f:
    json.dump(meta, f, ensure_ascii=False)

In [ ]:
import os, glob, json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1) load docs: assume each file in data/kb_docs/*.txt
docs = []
meta = []
for path in glob.glob(r"C:\Users\Lenovo\Ai_farmer_query_based\data\kb_docs\*.txt"):
    with open(path, 'r', encoding='utf8') as f:
        txt = f.read()
    # chunking (simple)
    chunk_size = 500
    for i in range(0, len(txt), chunk_size):
        chunk = txt[i:i+chunk_size]
        docs.append(chunk)
        meta.append({"source": os.path.basename(path), "pos": i})

# 2) embed them
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # multilingual small model
emb = model.encode(docs, convert_to_numpy=True, show_progress_bar=True)

# 3) normalize and build FAISS index (inner product)
faiss.normalize_L2(emb)
dim = emb.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(emb)

# 4) persist index & meta
faiss.write_index(index, "data/faiss_index.idx")
with open("data/faiss_meta.json", "w", encoding='utf8') as f:
    json.dump(meta, f, ensure_ascii=False)
